# Load nflverse data

## Overview


A Python package for downloading NFL data from nflverse repositories. This is a Python port of the popular R package nflreadr, designed to provide easy access to NFL data with caching, progress tracking, and modern Python conventions.

**Features**:¶

* Compatible API with nflreadr R package
* Fast data loading with Polars DataFrames
* Intelligent caching (memory or filesystem)
* Progress tracking for large downloads

**Links**:

* Main page: https://nflreadpy.nflverse.com/
* Load functions: https://nflreadpy.nflverse.com/api/load_functions/
* Configuration: https://nflreadpy.nflverse.com/api/configuration/
* Utilities: https://nflreadpy.nflverse.com/api/utils/

Examples of using the nflreadpy library:

```python
import nflreadpy as nfl

# Load current season play-by-play data
pbp = nfl.load_pbp()

# Load player game-level stats for multiple seasons
player_stats = nfl.load_player_stats([2022, 2023])

# Load all available team level stats
team_stats = nfl.load_team_stats(seasons=True)

# nflreadpy uses Polars instead of pandas. Convert to pandas if needed:
pbp_pandas = pbp.to_pandas()
```

## Config

In [75]:
import nflreadpy as nfl
import polars as pl

In [47]:
''' Utility Functions
get_current_season(roster: bool = False) -> int
get_current_week() -> int
'''

# Set current season
cur_season = nfl.get_current_season()
print(f"Current season: {cur_season}")

# Set current week
cur_week = nfl.get_current_week()
print(f"Current week: {cur_week}")

Current season: 2025
Current week: 3


## Helper Functions

In [98]:
''' Note: These functions have not been tested with
functions other than load player stats
'''

# Renames player_id column to gsis_id to match actual name of the id
def rename_col_player_id_to_gsis_id(df):
    return df.rename({"player_id": "gsis_id"})

# Uses summary level to help with file name
def set_file_suffix(summary_level):
    if summary_level == "week":
        file_suffix = "wkly"
    elif summary_level == "reg+post":
        file_suffix = "full"
    else:
        file_suffix = summary_level
    return file_suffix

# Saves df to csv
def save_csv(df, stat_type, season, grouping):
    file_suffix = set_file_suffix(grouping)
    file_name = f"{stat_type}_{season}_{file_suffix}.csv"
    df.write_csv(file_name, separator=",", include_header=True)
    print(f"Wrote DataFrame to '{file_name}'.")

## Player NFL Statistics

### Load Player Statistics

```python
load_player_stats(
    seasons: int | list[int] | bool | None = None,
    summary_level: Literal[
        "week", "reg", "post", "reg+post"
    ] = "week",
) -> pl.DataFrame
```

#### Load Player Stats for Past Seasons

In [74]:
''' TODO: Improve CSV output to allow for specifying output directory and to
organize by season
'''

# Save historical player statistics by season from list of seasons
def save_player_stats_hist(season_list, summary_level):
    for year in season_list:
        season = year
        # Get player stats for past season by week
        plr_stats = nfl.load_player_stats(season, summary_level)
        # Rename 'player_id' column to 'nfl_id' to match core.player_id table schema
        plr_stats = rename_col_player_id(plr_stats)
        # Save to csv file
        save_csv(plr_stats, "plr_stats", season, summary_level)
    return

if __name__ == "__main__":

    # Define list of seasons
    hist_seasons_plr_stats = [
        2022,
        2021,
        2020,
        2019,
        2018,
        2017,
        2016,
        2015
]
    # Save by week
    save_player_stats_hist(hist_seasons_plr_stats, "week")
    
    # Save by regular season
    save_player_stats_hist(hist_seasons_plr_stats, "reg")

    # Save by post season
    save_player_stats_hist(hist_seasons_plr_stats, "post")

    # Save by full season
    save_player_stats_hist(hist_seasons_plr_stats, "reg+post")


Wrote DataFrame to 'plr_stats_2022_wkly.csv'.
Wrote DataFrame to 'plr_stats_2021_wkly.csv'.
Wrote DataFrame to 'plr_stats_2020_wkly.csv'.
Wrote DataFrame to 'plr_stats_2019_wkly.csv'.
Wrote DataFrame to 'plr_stats_2018_wkly.csv'.
Wrote DataFrame to 'plr_stats_2017_wkly.csv'.
Wrote DataFrame to 'plr_stats_2016_wkly.csv'.
Wrote DataFrame to 'plr_stats_2015_wkly.csv'.
Wrote DataFrame to 'plr_stats_2022_reg.csv'.
Wrote DataFrame to 'plr_stats_2021_reg.csv'.
Wrote DataFrame to 'plr_stats_2020_reg.csv'.
Wrote DataFrame to 'plr_stats_2019_reg.csv'.
Wrote DataFrame to 'plr_stats_2018_reg.csv'.
Wrote DataFrame to 'plr_stats_2017_reg.csv'.
Wrote DataFrame to 'plr_stats_2016_reg.csv'.
Wrote DataFrame to 'plr_stats_2015_reg.csv'.
Wrote DataFrame to 'plr_stats_2022_post.csv'.
Wrote DataFrame to 'plr_stats_2021_post.csv'.
Wrote DataFrame to 'plr_stats_2020_post.csv'.
Wrote DataFrame to 'plr_stats_2019_post.csv'.
Wrote DataFrame to 'plr_stats_2018_post.csv'.
Wrote DataFrame to 'plr_stats_2017_post.cs

#### Load Player Stats for Current Season

In [48]:
''' TODO: Enhance this with function using approach 
in load historical player stats above
'''

# Get current season player stats by week
plr_stats_cur_season_wkly = nfl.load_player_stats(cur_season, "week")
# Save current season player stats by week to csv
save_csv(plr_stats_cur_season_wkly, f"plr_stats_wkly_{cur_season}-{cur_week}.csv")

# Get current season player stats by regular season
plr_stats_cur_season_reg = nfl.load_player_stats(cur_season, "reg")
# Save current season player stats by regular season to csv
save_csv(plr_stats_cur_season_reg, f"plr_stats_reg_{cur_season}-{cur_week}.csv")

Wrote DataFrame to 'plr_stats_wkly_2025-3.csv'.
Wrote DataFrame to 'plr_stats_reg_2025-3.csv'.


### Load NextGen Stats

```python
load_nextgen_stats(
    seasons: int | list[int] | bool | None = None,
    stat_type: Literal[
        "passing", "receiving", "rushing"
    ] = "passing",
) -> pl.DataFrame
```

### Load Participation Data

```python
load_participation(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```

**Note:** Data available since 2016.

### Load Injury Data

```python
load_injuries(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```

### Load Snap Counts

```python
load_snap_counts(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```

## Play-By-Play NFL Statistics

### Load NFL play-by-play data

```python
load_pbp(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```

### Load FTN Manual Play Charting Data

```python
load_ftn_charting(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```

**Note**: Data available since 2022

## Team NFL Stats

## Fantasy-Related Information

### Load Fantasy Football Player IDs

```python
load_ff_playerids() -> pl.DataFrame
```

In [ ]:
# Loads fantasy football player IDs from DynastyProcess.com database
ff_playerids = nfl.load_ff_playerids()
ff_playerids.sample(15)

### Load fantasy football rankings and projections

```python
load_ff_rankings(
    type: Literal["draft", "week", "all"] = "draft",
) -> pl.DataFrame
```

**Paremters**:

| Name | Type | Description | Default |
|:--|:--|:--|:--|
| type | Literal['draft', 'week', 'all'] | Type of rankings to load: - "draft": Draft rankings/projections - "week": Weekly rankings/projections - "all": All historical rankings/projections | 'draft' |

**Returns**:

| Type | Description |
|:--|:--|
| DataFrame | Polars DataFrame with fantasy football rankings data. |

**See also**: https://nflreadr.nflverse.com/reference/load_ff_rankings.html


In [ ]:
# Load the latest fantasy football rankings and projections
ff_rankings = nfl.load_ff_rankings("week")
ff_rankings.sample(10)

In [ ]:
# Load the preseason fantasy football rankings and projections
ff_rankings = nfl.load_ff_rankings("draft")
ff_rankings.sample(10)

In [ ]:
# Load the full historical archive of fantasy football rankings and projections
ff_rankings = nfl.load_ff_rankings("all")
ff_rankings.sample(10)

### Load FF Opportunities

```python
load_ff_opportunity(
    seasons: int | list[int] | None = None,
    stat_type: Literal[
        "weekly", "pbp_pass", "pbp_rush"
    ] = "weekly",
    model_version: Literal["latest", "v1.0.0"] = "latest",
) -> pl.DataFrame
```

This function loads opportunity and target share data for fantasy football analysis from the ffverse/ffopportunity repository.

In [91]:
# Load weekly fantasy football opportunity data for current season
ff_opportunity_current_wkly = nfl.load_ff_opportunity()

In [93]:
# Load historical fantasy football opportunity data

opportunity_data_seasons = [
    2024,
    2023,
    2022,
    2021,
    2020
]

ff_opportunity_2020_to_2024 = nfl.load_ff_opportunity(opportunity_data_seasons)

## Other Information

### Load Player Information

```python
load_players() -> pl.DataFrame
```

This is a comprehensive source of player information including basic details, draft information, positions, and ID mappings across multiple data sources (GSIS, PFR, PFF, OTC, ESB, ESPN).

In [97]:
# Load player information

player_data = nfl.load_players()
player_data.describe()
player_data.sample(15)

gsis_id,display_name,common_first_name,first_name,last_name,short_name,football_name,suffix,esb_id,nfl_id,pfr_id,pff_id,otc_id,espn_id,smart_id,birth_date,position_group,position,ngs_position_group,ngs_position,height,weight,headshot,college_name,college_conference,jersey_number,rookie_season,last_season,latest_team,status,ngs_status,ngs_status_short_description,years_of_experience,pff_position,pff_status,draft_year,draft_round,draft_pick,draft_team
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,str,str,str,str,i32,i32,str,str,str,str,i32,str,str,i32,i32,i32,str
"""00-0034807""","""Ogbo Okoronkwo""","""Ogbo""","""Ogbonnia""","""Okoronkwo""","""O.Okoronkwo""","""Ogbonnia""",null,"""OKO385924""","""46229""","""OkorOg00""","""50130""","""7045""","""3052667""","""32004f4b-4f38-5924-fae5-ebda75…","""1995-04-24""","""DL""","""DE""","""DL""","""EDGE""",74,253,"""https://static.www.nfl.com/ima…","""Oklahoma""","""Big Twelve Conference""","""45""",2018,2025,"""PHI""","""ACT""","""ACT""","""Active""",8,"""ED""","""A""",2018,5,160,"""LA"""
"""00-0020504""","""Derrick Burgess""","""Derrick""","""Derrick""","""Burgess""",null,null,null,"""BUR072504""","""26281""","""BurgDe99""","""830""",null,"""2611""","""32004255-5207-2504-d249-927c11…","""1978-08-12""","""DL""","""DE""",null,null,74,255,"""https://static.www.nfl.com/ima…","""Mississippi""",null,"""73""",2001,2010,"""PHI""","""CUT""",null,null,10,null,null,2001,3,63,"""PHI"""
"""00-0039704""","""Matthew Cindric""","""Matthew""","""Matthew""","""Cindric""","""M.Cindric""","""Matthew""",null,"""CIN415291""","""57867""",null,"""81604""","""12120""","""4372738""","""32004349-4e41-5291-9802-eeb610…","""1999-08-02""","""OL""","""C""","""OL""","""G""",76,295,null,"""California""","""Pacific Twelve Conference""",null,2024,2024,"""NYJ""","""DEV""","""CUT""","""Waivers/No Rec.""",0,null,null,null,null,null,null
"""00-0031427""","""Todd Davis""","""Todd""","""Todd""","""Davis""","""T.Davis""","""Todd""",null,"""DAV768914""","""42138""","""DaviTo01""","""9007""","""3680""","""17497""","""32004441-5676-8914-3a6a-2ae692…","""1992-05-17""","""LB""","""ILB""",null,null,73,230,"""https://static.www.nfl.com/ima…","""Sacramento State""","""Big Sky Conference""","""40""",2014,2020,"""MIN""","""ACT""","""RET""","""R/Retired""",7,"""LB""",null,null,null,null,null
"""00-0031728""","""Zach Vigil""","""Zach""","""Zachary""","""Vigil""","""Z.Vigil""","""Zach""",null,"""VIG259556""","""42759""","""VigiZa00""","""9824""","""4179""","""2517389""","""32005649-4725-9556-a89e-c5b26b…","""1991-03-28""","""LB""","""LB""","""LB""","""MLB""",74,238,"""https://static.www.nfl.com/ima…","""Utah State""","""Mountain West Conference""","""56""",2015,2018,"""WAS""","""ACT""","""ACT""","""Active""",4,"""LB""",null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""00-0014869""","""Ricky Siglar""","""Ricky""","""Ricky""","""Siglar""",null,null,null,"""SIG332232""",null,"""SiglRi20""",null,null,"""7404""","""32005349-4733-2232-4a3a-b912ff…","""1966-06-14""","""OL""","""OT""",null,null,79,308,"""https://static.www.nfl.com/ima…","""San Jose State; Arizona Wester…",null,"""72""",1990,1998,"""KC""","""ACT""",null,null,7,null,null,null,null,null,null
"""00-0027891""","""Golden Tate""","""Golden""","""Golden""","""Tate""","""G.Tate""","""Golden""",null,"""TAT245022""","""35499""","""TateGo00""","""5585""","""1541""","""13217""","""32005441-5424-5022-18f3-b9b543…","""1988-08-02""","""WR""","""WR""",null,null,71,191,"""https://static.www.nfl.com/ima…","""Notre Dame""","""Independent""","""0""",2010,2021,"""TEN""","""CUT""","""CUT""","""PS; Vet""",12,"""WR""",null,2010,2,60,"""SEA"""
"""00-0015243""","""Lance Smith""","""Lance""","""Lance""","""Smith""",null,null,null,"""SMI464000""",null,"""SmitLa20""",null,null,"""6901""","""3200534d-4946-4000-7d43-9350af…","""1963-01-01""","""OL""","""G""",null,null,75,282,"""https://static.www.nfl.com/ima…","""LSU

### Load Rosters

### Load NFL Combine Data

```python
load_combine(
    seasons: int | list[int] | bool | None = True,
) -> pl.DataFrame
```

### Load NFL Depth Charts

```python
load_depth_charts(
    seasons: int | list[int] | bool | None = None,
) -> pl.DataFrame
```